In [1]:
#NOTE: use paimg1 env, the retccl one has package issue with torchvision
import sys
import os
import numpy as np
import openslide
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Agg')
import pandas as pd
import warnings
import torch
import torch.nn as nn

from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader
import torch.optim as optim
from pathlib import Path

sys.path.insert(0, '../Utils/')
from Utils import create_dir_if_not_exists
from Utils import generate_deepzoom_tiles, extract_tile_start_end_coords, get_map_startend
from Utils import get_downsample_factor
from Utils import minmax_normalize
from Utils import log_message, set_seed
from Eval import compute_performance, plot_LOSS, compute_performance_each_label, get_attention_and_tileinfo
from train_utils import pull_tiles, get_feature_label_array_dynamic
from train_utils import ModelReadyData_diffdim, convert_to_dict, prediction
from Model import Mutation_MIL_MT
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
####################################
######      USERINPUT       ########
####################################
SELECTED_LABEL = ["AR","MMR (MSH2, MSH6, PMS2, MLH1, MSH3, MLH3, EPCAM)2","PTEN","RB1","TP53","TMB_HIGHorINTERMEDITATE","MSI_POS"]
#SELECTED_FEATURE = [str(i) for i in range(0,2048)] + ['TUMOR_PIXEL_PERC']
SELECTED_FEATURE = [str(i) for i in range(0,2048)]
TUMOR_FRAC_THRES = 0
TRAIN_SAMPLE_SIZE = "ALL_TUMOR_TILES"
TRAIN_OVERLAP = 100
TEST_OVERLAP = 0


##################
###### DIR  ######
##################
proj_dir = '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/'
wsi_path = proj_dir + '/data/OPX/'
label_path = proj_dir + 'data/MutationCalls/'
ft_ids_path =  proj_dir + 'intermediate_data/cd_finetune/cancer_detection_training/' #the ID used for fine-tuning cancer detection model, needs to be excluded from mutation study
train_tile_path = proj_dir + 'intermediate_data/cancer_prediction_results110224/IMSIZE250_OL' + str(TRAIN_OVERLAP) + '/'
test_tile_path =  proj_dir + 'intermediate_data/cancer_prediction_results110224/IMSIZE250_OL' + str(TEST_OVERLAP) + '/'
feature_name = 'features_alltiles_retccl'

################################################
#Create output dir
################################################
outdir =   proj_dir + 'intermediate_data/model_ready_data/' +'MAX_SS'+ str(TRAIN_SAMPLE_SIZE) + '_NFEATURES' + str(len(SELECTED_FEATURE)) + '_TrainOL' + str(TRAIN_OVERLAP) +  '_TestOL' + str(TEST_OVERLAP) + '/'
create_dir_if_not_exists(outdir)

##################
#Select GPU
##################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
set_seed(0)

Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/model_ready_data/MAX_SSALL_TUMOR_TILES_NFEATURES2048_TrainOL100_TestOL0/' already exists.
cuda:0


In [3]:
############################################################################################################
#Select IDS
############################################################################################################
#All available IDs
opx_ids = [x.replace('.tif','') for x in os.listdir(wsi_path)] #207
opx_ids.sort()

#Get IDs that are in FT train or already processed to exclude 
ft_ids_df = pd.read_csv(ft_ids_path + 'all_tumor_fraction_info.csv')
ft_train_ids = list(ft_ids_df.loc[ft_ids_df['Train_OR_Test'] == 'Train','sample_id'])

#OPX_182 –Exclude Possible Colon AdenoCa 
toexclude_ids = ft_train_ids + ['OPX_182']  #25


#Exclude ids in ft_train or processed
selected_ids = [x for x in opx_ids if x not in toexclude_ids] #199
print(len(selected_ids))

199


In [4]:
############################################################################################################
#Get Train and test IDs, 80% - 20%
############################################################################################################
# Number of folds
n_splits = 5
selected_fold = 0

# Initialize KFold
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Generate the folds
train_ids_folds = []
test_ids_folds = []
for fold, (train_index, test_index) in enumerate(kf.split(selected_ids)):
    train_ids_folds.append([selected_ids[i] for i in train_index])
    test_ids_folds.append([selected_ids[i] for i in test_index])

full_train_ids = train_ids_folds[selected_fold]
test_ids = test_ids_folds[selected_fold]

# Randomly select 5% of the train_ids for validation
train_ids, val_ids = train_test_split(full_train_ids, test_size=0.05, random_state=42)
print(len(train_ids))
print(len(val_ids))
print(len(test_ids))

151
8
40


In [5]:
############################################################################################################
#Get features and labels
#NOTE: OPX_005 has no tumor tiles, so excluded in this step
############################################################################################################
train_feature, train_label, train_info, train_tf_info, selected_train_ids = get_feature_label_array_dynamic(train_tile_path,feature_name, train_ids, SELECTED_LABEL,SELECTED_FEATURE,"Train" ,tumor_fraction_thres = TUMOR_FRAC_THRES,train_sample_size = TRAIN_SAMPLE_SIZE)
val_feature, val_label, val_info, val_tf_info, select_val_ids = get_feature_label_array_dynamic(train_tile_path,feature_name, val_ids, SELECTED_LABEL,SELECTED_FEATURE, "Train" ,tumor_fraction_thres = TUMOR_FRAC_THRES,train_sample_size = TRAIN_SAMPLE_SIZE)
test_feature, test_label, test_info, test_tf_info, select_test_ids = get_feature_label_array_dynamic(test_tile_path, feature_name, test_ids, SELECTED_LABEL, SELECTED_FEATURE, "Test", tumor_fraction_thres = 0)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
0
0
10
20
30


In [6]:
# Count the number of 1s in each column
train_label_np = np.concatenate(train_label)
count_ones = np.sum(train_label_np == 1, axis=0)

print("Number of 1s in each column:", count_ones)
percentage_ones = np.round((count_ones / train_label_np.shape[0]) * 100,1)
print("% of 1s in each column:", percentage_ones)
print(["Mutation labels  :","AR","HR","PTEN","RB1","TP53","TMB","MSI_POS"])

# Count the number of 1s in each column
test_label_np = np.concatenate(test_label)
count_ones = np.sum(test_label_np == 1, axis=0)

print("--------TEST------")
print("Number of 1s in each column:", count_ones)
percentage_ones = np.round((count_ones / test_label_np.shape[0]) * 100,1)
print("% of 1s in each column:", percentage_ones)
print(["Mutation labels  :","AR","HR","PTEN","RB1","TP53","TMB","MSI_POS"])

Number of 1s in each column: [16 25 28 17 56  5  4]
% of 1s in each column: [10.7 16.7 18.7 11.3 37.3  3.3  2.7]
['Mutation labels  :', 'AR', 'HR', 'PTEN', 'RB1', 'TP53', 'TMB', 'MSI_POS']
--------TEST------
Number of 1s in each column: [ 5  6  9  4 16  3  3]
% of 1s in each column: [12.5 15.  22.5 10.  40.   7.5  7.5]
['Mutation labels  :', 'AR', 'HR', 'PTEN', 'RB1', 'TP53', 'TMB', 'MSI_POS']


In [8]:
################################################
#     Model ready data 
################################################
train_data = ModelReadyData_diffdim(train_feature,train_label,train_tf_info)
test_data = ModelReadyData_diffdim(test_feature,test_label,test_tf_info)
val_data = ModelReadyData_diffdim(val_feature,val_label,val_tf_info)

#Output
torch.save(train_data, outdir + 'train_data.pth')
torch.save(test_data,  outdir + 'test_data.pth')
torch.save(val_data,   outdir + 'val_data.pth')

torch.save(train_info,   outdir + 'train_info.pth')
torch.save(test_info,   outdir + 'test_info.pth')
torch.save(val_info,   outdir + 'val_info.pth')

torch.save(train_tf_info,   outdir + 'train_tf_info.pth')
torch.save(test_tf_info,   outdir + 'test_tf_info.pth')
torch.save(val_tf_info,   outdir + 'val_tf_info.pth')


torch.save(selected_train_ids,   outdir + 'train_ids.pth')
torch.save(select_test_ids,   outdir + 'test_ids.pth')
torch.save(select_val_ids,   outdir + 'val_ids.pth')